In [7]:
# -------------------- Standard Library Imports --------------------
import argparse
import glob
import os
import sys
from multiprocessing import Pool, cpu_count
from random import sample
from subprocess import PIPE, Popen, STDOUT, call, run

# -------------------- Scientific Libraries ------------------------
import numpy as np
import pandas as pd
import pybedtools
import pysam
import scipy
import statsmodels.stats.multitest as smm
from Bio import SeqIO

pybedtools.helpers.set_tempdir('/fs/cbsuhy02/storage/jz855/tmp/') 


In [2]:
# Specify root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(project_root)


/local/storage/jz855/STARR_seq_code/Final_Code_Sharing


# LentiMPRA

In [22]:
# Read the input gzipped BED file containing all tested elements in both orientations
input_file = os.path.join(project_root, 'data', 'uniform_processed_data', 
                          'LentiMPRA', 'element_level', 'all_element_tested_in_both_orientations.bed.gz')
lenti_tested_regions = pd.read_csv(input_file, sep='\t', header=None)
print("Number of tested regions (all orientations):", len(lenti_tested_regions))

# Read the orientation-independent peak file
peak_file = os.path.join(project_root, 'data', 'uniform_processed_data', 
                          'LentiMPRA', 'merged_peak', 'merged_enhancer_peak_orientation_independent.bed.gz')
lenti_orientation_indep_peak = pd.read_csv(peak_file, sep='\t', header=None)
print("Number of orientation-independent peaks:", len(lenti_orientation_indep_peak))

# Convert orientation-independent peak DataFrame to a BedTool object
lenti_orientation_indep_peak_bed = pybedtools.BedTool.from_dataframe(lenti_orientation_indep_peak)

# -------------------------------------------------------------------------
# Filter regions:
#   - Exclude regions where the value in column 19 is 'enhancer'
#     (indicating they are active in both orientations).
#   - Retain regions where either column 7 or column 13 is 'enhancer'
#     (i.e. only one orientation is active).
# -------------------------------------------------------------------------
lenti_tested_regions = lenti_tested_regions[lenti_tested_regions[19] != 'enhancer']
lenti_either_active_regions = lenti_tested_regions[
    (lenti_tested_regions[7] == 'enhancer') | (lenti_tested_regions[13] == 'enhancer')
]
print("Number of active regions (only one orientation active):", len(lenti_either_active_regions))

# -------------------------------------------------------------------------
# Convert the filtered DataFrame to a BedTool object for merging overlapping intervals.
# -------------------------------------------------------------------------
lenti_either_active_regions_bedtool = pybedtools.BedTool.from_dataframe(lenti_either_active_regions)
print("Number of regions in BedTool object (after filtering):", len(lenti_either_active_regions_bedtool))

# -------------------------------------------------------------------------
# Exclude regions overlapping with orientation-independent peaks (>=90% overlap)
# -------------------------------------------------------------------------
lenti_either_active_regions_overlap_with_orientation_indep_peak = lenti_either_active_regions_bedtool.intersect(lenti_orientation_indep_peak_bed, wa=True, f=0.9)
print("Number of regions overlapping with orientation-independent peaks (>=90%):", len(lenti_either_active_regions_overlap_with_orientation_indep_peak))

lenti_either_active_regions_bedtool = lenti_either_active_regions_bedtool.intersect(lenti_either_active_regions_overlap_with_orientation_indep_peak, v=True, F=1)
print("Number of regions remaining after excluding overlaps:", len(lenti_either_active_regions_bedtool))

# -------------------------------------------------------------------------
# Merge overlapping regions in the BedTool and convert back to a DataFrame.
# -------------------------------------------------------------------------
lenti_merged_active_regions = lenti_either_active_regions_bedtool.merge().to_dataframe(disable_auto_names=True, header=None)
print("Number of merged active regions:", len(lenti_merged_active_regions))

# -------------------------------------------------------------------------
# Write the merged active regions to an output gzipped BED file.
# -------------------------------------------------------------------------
output_file = os.path.join(project_root, 'data', 'uniform_processed_data', 
                           'LentiMPRA', 'merged_peak', 'merged_enhancer_peak_from_either_in_tested_both.bed.gz')
lenti_merged_active_regions.to_csv(output_file, sep='\t', header=False, index=False)
print("Finished writing merged active regions to:", output_file)


Number of tested regions (all orientations): 113673
Number of orientation-independent peaks: 16603
Number of active regions (only one orientation active): 3875
Number of regions in BedTool object (after filtering): 3875
Number of regions overlapping with orientation-independent peaks (>=90%): 1
Number of regions remaining after excluding overlaps: 3874
Number of merged active regions: 3872
Finished writing merged active regions to: /local/storage/jz855/STARR_seq_code/Final_Code_Sharing/data/uniform_processed_data/LentiMPRA/merged_peak/merged_enhancer_peak_from_either_in_tested_both.bed.gz


# ATAC-STARR-seq

In [32]:
# Read the input gzipped BED file containing all tested elements in both orientations
input_file = os.path.join(project_root, 'data', 'uniform_processed_data', 
                          'ATAC_STARR_seq', 'bin_level', 'all_bin_tested_in_both_orientations.bed.gz')
atac_starr_tested_regions = pd.read_csv(input_file, sep='\t', header=None)
print("Number of tested regions (all orientations):", len(atac_starr_tested_regions))

# Read the orientation-independent peak file
peak_file = os.path.join(project_root, 'data', 'uniform_processed_data', 
                          'ATAC_STARR_seq', 'merged_peak', 'merged_enhancer_peak_orientation_independent.bed.gz')
atac_starr_orientation_indep_peak = pd.read_csv(peak_file, sep='\t', header=None)
print("Number of orientation-independent peaks:", len(atac_starr_orientation_indep_peak))

# Convert orientation-independent peak DataFrame to a BedTool object
atac_starr_orientation_indep_peak_bed = pybedtools.BedTool.from_dataframe(atac_starr_orientation_indep_peak)

# -------------------------------------------------------------------------
# Filter regions:
#   - Exclude regions where the value in column 19 is 'enhancer'
#     (indicating they are active in both orientations).
#   - Retain regions where either column 7 or column 13 is 'enhancer'
#     (i.e. only one orientation is active).
# -------------------------------------------------------------------------
atac_starr_tested_regions = atac_starr_tested_regions[atac_starr_tested_regions[19] != 'enhancer']
atac_starr_either_active_regions = atac_starr_tested_regions[
    (atac_starr_tested_regions[7] == 'enhancer') | (atac_starr_tested_regions[13] == 'enhancer')
]
print("Number of active regions (only one orientation active):", len(atac_starr_either_active_regions))

# -------------------------------------------------------------------------
# Convert the filtered DataFrame to a BedTool object for merging overlapping intervals.
# -------------------------------------------------------------------------
atac_starr_either_active_regions_bedtool = pybedtools.BedTool.from_dataframe(atac_starr_either_active_regions)
print("Number of regions in BedTool object (after filtering):", len(atac_starr_either_active_regions_bedtool))

# -------------------------------------------------------------------------
# Exclude regions overlapping with orientation-independent peaks (>=90% overlap)
# -------------------------------------------------------------------------
# atac_starr_either_active_regions_overlap_with_orientation_indep_peak = atac_starr_either_active_regions_bedtool.intersect(atac_starr_orientation_indep_peak_bed, wa=True, f=0.9)
# print("Number of regions overlapping with orientation-independent peaks (>=90%):", len(atac_starr_either_active_regions_overlap_with_orientation_indep_peak))

# atac_starr_either_active_regions_bedtool = atac_starr_either_active_regions_bedtool.intersect(atac_starr_either_active_regions_overlap_with_orientation_indep_peak, v=True, F=1)

atac_starr_either_active_regions_bedtool = atac_starr_either_active_regions_bedtool.intersect(atac_starr_orientation_indep_peak_bed, v=True)
print("Number of regions remaining after excluding overlaps:", len(atac_starr_either_active_regions_bedtool))

# -------------------------------------------------------------------------
# Merge overlapping regions in the BedTool and convert back to a DataFrame.
# -------------------------------------------------------------------------
atac_starr_merged_active_regions = atac_starr_either_active_regions_bedtool.merge().to_dataframe(disable_auto_names=True, header=None)
print("Number of merged active regions:", len(atac_starr_merged_active_regions))

# -------------------------------------------------------------------------
# Write the merged active regions to an output gzipped BED file.
# -------------------------------------------------------------------------
output_file = os.path.join(project_root, 'data', 'uniform_processed_data', 
                           'ATAC_STARR_seq', 'merged_peak', 'merged_enhancer_peak_from_either_in_tested_both.bed.gz')
atac_starr_merged_active_regions.to_csv(output_file, sep='\t', header=False, index=False)
print("Finished writing merged active regions to:", output_file)



Number of tested regions (all orientations): 10473381
Number of orientation-independent peaks: 11679
Number of active regions (only one orientation active): 103197
Number of regions in BedTool object (after filtering): 103197
Number of regions remaining after excluding overlaps: 59851
Number of merged active regions: 16421
Finished writing merged active regions to: /local/storage/jz855/STARR_seq_code/Final_Code_Sharing/data/uniform_processed_data/ATAC_STARR_seq/merged_peak/merged_enhancer_peak_from_either_in_tested_both.bed.gz


In [34]:
# -------------------------------------------------------------------------
# Convert the merged active regions DataFrame to a BedTool object.
# -------------------------------------------------------------------------
atac_starr_merged_active_regions_bed = pybedtools.BedTool.from_dataframe(atac_starr_merged_active_regions)

# Print the number of intervals in the merged active regions BedTool.
print("Number of merged active regions (BedTool):", len(atac_starr_merged_active_regions_bed))

# Print the number of intervals in the orientation-independent peak BedTool.
print("Number of orientation-independent peaks (BedTool):", len(atac_starr_orientation_indep_peak_bed))

# -------------------------------------------------------------------------
# Compute the intersection between the orientation-independent peaks and the 
# merged active regions. The 'wao' option reports the number of overlapping 
# base pairs for each feature (adds a column at index 17).
# -------------------------------------------------------------------------
intersect = atac_starr_orientation_indep_peak_bed.intersect(atac_starr_merged_active_regions_bed, wao=True).to_dataframe(disable_auto_names=True, header=None)

# Print the number of rows in the intersection DataFrame.
print("Number of intersection entries:", len(intersect))

# -------------------------------------------------------------------------
# Sort the intersection DataFrame based on the overlap column (column index 17).
# -------------------------------------------------------------------------
intersect = intersect.sort_values(17)

# Print the number of entries where there is no overlap (overlap equals zero).
print("Number of intersections with zero overlap:", len(intersect[intersect[17] == 0]))

# -------------------------------------------------------------------------
# Identify the entries with a positive overlap.
# -------------------------------------------------------------------------
have_intersect = intersect[intersect[17] > 0]

# Print the number of entries with a positive overlap.
print("Number of entries with positive overlap:", len(have_intersect))

# Print the descriptive statistics (e.g., count, mean, std, min, max, percentiles)
# for the overlap column (column index 17) among the entries with positive overlap.
print("Descriptive statistics of overlap (positive entries):")
print(have_intersect[17].describe())


Number of merged active regions (BedTool): 16421
Number of orientation-independent peaks (BedTool): 11679
Number of intersection entries: 11679
Number of intersections with zero overlap: 11679
Number of entries with positive overlap: 0
Descriptive statistics of overlap (positive entries):
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: 17, dtype: float64


# WHG-STARR-seq

In [35]:
# Read the input gzipped BED file containing all tested elements in both orientations
input_file = os.path.join(project_root, 'data', 'uniform_processed_data', 
                          'WHG_STARR_seq', 'bin_level', 'all_bin_tested_in_both_orientations.bed.gz')
whg_starr_tested_regions = pd.read_csv(input_file, sep='\t', header=None)
print("Number of tested regions (all orientations):", len(whg_starr_tested_regions))

# Read the orientation-independent peak file
peak_file = os.path.join(project_root, 'data', 'uniform_processed_data', 
                          'WHG_STARR_seq', 'merged_peak', 'merged_enhancer_peak_orientation_independent.bed.gz')
whg_starr_orientation_indep_peak = pd.read_csv(peak_file, sep='\t', header=None)
print("Number of orientation-independent peaks:", len(whg_starr_orientation_indep_peak))

# Convert orientation-independent peak DataFrame to a BedTool object
whg_starr_orientation_indep_peak_bed = pybedtools.BedTool.from_dataframe(whg_starr_orientation_indep_peak)

# -------------------------------------------------------------------------
# Filter regions:
#   - Exclude regions where the value in column 19 is 'enhancer'
#     (indicating they are active in both orientations).
#   - Retain regions where either column 7 or column 13 is 'enhancer'
#     (i.e. only one orientation is active).
# -------------------------------------------------------------------------
whg_starr_tested_regions = whg_starr_tested_regions[whg_starr_tested_regions[19] != 'enhancer']
whg_starr_either_active_regions = whg_starr_tested_regions[
    (whg_starr_tested_regions[7] == 'enhancer') | (whg_starr_tested_regions[13] == 'enhancer')
]
print("Number of active regions (only one orientation active):", len(whg_starr_either_active_regions))

# -------------------------------------------------------------------------
# Convert the filtered DataFrame to a BedTool object for merging overlapping intervals.
# -------------------------------------------------------------------------
whg_starr_either_active_regions_bedtool = pybedtools.BedTool.from_dataframe(whg_starr_either_active_regions)
print("Number of regions in BedTool object (after filtering):", len(whg_starr_either_active_regions_bedtool))

# -------------------------------------------------------------------------
# Exclude regions overlapping with orientation-independent peaks (>=90% overlap)
# -------------------------------------------------------------------------
# whg_starr_either_active_regions_overlap_with_orientation_indep_peak = whg_starr_either_active_regions_bedtool.intersect(whg_starr_orientation_indep_peak_bed, wa=True, f=0.9)
# print("Number of regions overlapping with orientation-independent peaks (>=90%):", len(whg_starr_either_active_regions_overlap_with_orientation_indep_peak))

# whg_starr_either_active_regions_bedtool = whg_starr_either_active_regions_bedtool.intersect(whg_starr_either_active_regions_overlap_with_orientation_indep_peak, v=True, F=1)
whg_starr_either_active_regions_bedtool = whg_starr_either_active_regions_bedtool.intersect(whg_starr_orientation_indep_peak_bed, v=True)
print("Number of regions remaining after excluding overlaps:", len(whg_starr_either_active_regions_bedtool))

# -------------------------------------------------------------------------
# Merge overlapping regions in the BedTool and convert back to a DataFrame.
# -------------------------------------------------------------------------
whg_starr_merged_active_regions = whg_starr_either_active_regions_bedtool.merge().to_dataframe(disable_auto_names=True, header=None)
print("Number of merged active regions:", len(whg_starr_merged_active_regions))

# -------------------------------------------------------------------------
# Write the merged active regions to an output gzipped BED file.
# -------------------------------------------------------------------------
output_file = os.path.join(project_root, 'data', 'uniform_processed_data', 
                           'WHG_STARR_seq', 'merged_peak', 'merged_enhancer_peak_from_either_in_tested_both.bed.gz')
whg_starr_merged_active_regions.to_csv(output_file, sep='\t', header=False, index=False)
print("Finished writing merged active regions to:", output_file)


Number of tested regions (all orientations): 109842059
Number of orientation-independent peaks: 25505
Number of active regions (only one orientation active): 447558
Number of regions in BedTool object (after filtering): 447558
Number of regions remaining after excluding overlaps: 282376
Number of merged active regions: 47189
Finished writing merged active regions to: /local/storage/jz855/STARR_seq_code/Final_Code_Sharing/data/uniform_processed_data/WHG_STARR_seq/merged_peak/merged_enhancer_peak_from_either_in_tested_both.bed.gz


In [36]:
# -------------------------------------------------------------------------
# Convert the merged active regions DataFrame to a BedTool object.
# -------------------------------------------------------------------------
whg_starr_merged_active_regions_bed = pybedtools.BedTool.from_dataframe(whg_starr_merged_active_regions)

# Print the number of intervals in the merged active regions BedTool.
print("Number of merged active regions (BedTool):", len(whg_starr_merged_active_regions_bed))

# Print the number of intervals in the orientation-independent peak BedTool.
print("Number of orientation-independent peaks (BedTool):", len(whg_starr_orientation_indep_peak_bed))

# -------------------------------------------------------------------------
# Compute the intersection between the orientation-independent peaks and the 
# merged active regions. The 'wao' option reports the number of overlapping 
# base pairs for each feature (adds a column at index 17).
# -------------------------------------------------------------------------
intersect = whg_starr_orientation_indep_peak_bed.intersect(whg_starr_merged_active_regions_bed, wao=True).to_dataframe(disable_auto_names=True, header=None)

# Print the number of rows in the intersection DataFrame.
print("Number of intersection entries:", len(intersect))

# -------------------------------------------------------------------------
# Sort the intersection DataFrame based on the overlap column (column index 17).
# -------------------------------------------------------------------------
intersect = intersect.sort_values(17)

# Print the number of entries where there is no overlap (overlap equals zero).
print("Number of intersections with zero overlap:", len(intersect[intersect[17] == 0]))

# -------------------------------------------------------------------------
# Identify the entries with a positive overlap.
# -------------------------------------------------------------------------
have_intersect = intersect[intersect[17] > 0]

# Print the number of entries with a positive overlap.
print("Number of entries with positive overlap:", len(have_intersect))

# Print the descriptive statistics (e.g., count, mean, std, min, max, percentiles)
# for the overlap column (column index 17) among the entries with positive overlap.
print("Descriptive statistics of overlap (positive entries):")
print(have_intersect[17].describe())


Number of merged active regions (BedTool): 47189
Number of orientation-independent peaks (BedTool): 25505
Number of intersection entries: 25505
Number of intersections with zero overlap: 25505
Number of entries with positive overlap: 0
Descriptive statistics of overlap (positive entries):
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: 17, dtype: float64


# TilingMPRA

In [40]:
 # -------------------- TilingMPRA OL13 --------------------

data_path = os.path.join(project_root, 'data', 'uniform_processed_data', 'TilingMPRA', 'OL13_ENCSR394HXI', 'merged_peak', 'merged_enhancer_peak_in_either_orientation.bed.gz')
ol13_merged_active = pd.read_csv(data_path, sep='\t', header=None)
print(len(ol13_merged_active))

ol13_merged_active = ol13_merged_active[[0,1,2]]
ol13_merged_active[3] = 'TilingMPRA_ENCSR394HXI_active'

# -------------------- TilingMPRA OL45 --------------------

data_path = os.path.join(project_root, 'data', 'uniform_processed_data', 'TilingMPRA', 'OL45_ENCSR363XER', 'merged_peak', 'merged_enhancer_peak_in_either_orientation.bed.gz')
ol45_merged_active = pd.read_csv(data_path, sep='\t', header=None)
print(len(ol45_merged_active))

ol45_merged_active = ol45_merged_active[[0,1,2]]
ol45_merged_active[3] = 'TilingMPRA_ENCSR363XER_active'

# -------------------- TilingMPRA OL43 --------------------

data_path = os.path.join(project_root, 'data', 'uniform_processed_data', 'TilingMPRA', 'OL43_ENCSR917SFD', 'merged_peak', 'merged_enhancer_peak_in_either_orientation.bed.gz')
ol43_merged_active = pd.read_csv(data_path, sep='\t', header=None)
print(len(ol43_merged_active))

ol43_merged_active = ol43_merged_active[[0,1,2]]
ol43_merged_active[3] = 'TilingMPRA_ENCSR917SFD_active'

tilingMPRA_uniform_processed_active = pd.concat([ol13_merged_active, ol45_merged_active, ol43_merged_active], ignore_index=True, axis=0)
print(len(tilingMPRA_uniform_processed_active))

data_path = os.path.join(project_root, 'data', 'uniform_processed_data', 'TilingMPRA', 'all_TilingMPRA_merged_enhancer_peak_in_either_orientation.bed.gz')
tilingMPRA_uniform_processed_active.to_csv(data_path, sep='\t', header=False, index=False, compression='gzip')
print('finished')


136
3761
2117
6014
finished


In [41]:
# -------------------- TilingMPRA OL13 --------------------

data_path = os.path.join(project_root, 'data', 'lab_reported_data', 'processed_files', 'TilingMPRA', 'OL13_merged_enhancer_peaks_in_either_orientation.bed')
ol13_merged_active = pd.read_csv(data_path, sep='\t', header=None)
print(len(ol13_merged_active))

ol13_merged_active = ol13_merged_active[[0,1,2]]
ol13_merged_active[3] = 'TilingMPRA_ENCSR394HXI_active'


# -------------------- TilingMPRA OL45 --------------------

data_path = os.path.join(project_root, 'data', 'lab_reported_data', 'processed_files', 'TilingMPRA', 'OL45_merged_enhancer_peaks_in_either_orientation.bed')
ol45_merged_active = pd.read_csv(data_path, sep='\t', header=None)
print(len(ol45_merged_active))

ol45_merged_active = ol45_merged_active[[0,1,2]]
ol45_merged_active[3] = 'TilingMPRA_ENCSR363XER_active'


# -------------------- TilingMPRA OL43 --------------------

data_path = os.path.join(project_root, 'data', 'lab_reported_data', 'processed_files', 'TilingMPRA', 'OL43_merged_enhancer_peaks_in_either_orientation.bed')
ol43_merged_active = pd.read_csv(data_path, sep='\t', header=None)
print(len(ol43_merged_active))

ol43_merged_active = ol43_merged_active[[0,1,2]]
ol43_merged_active[3] = 'TilingMPRA_ENCSR917SFD_active'

tilingMPRA_lab_reported_active = pd.concat([ol13_merged_active, ol45_merged_active, ol43_merged_active], ignore_index=True, axis=0)
print(len(tilingMPRA_lab_reported_active))

data_path = os.path.join(project_root, 'data', 'lab_reported_data', 'processed_files', 'TilingMPRA', 'all_TilingMPRA_active_from_lab_reported_data.bed.gz')
tilingMPRA_lab_reported_active.to_csv(data_path, sep='\t', header=False, index=False, compression='gzip')
print('finished')


121
8379
4419
12919
finished
